## Ablation study


For our project we plan to reproduce the experiment with In Hospital Mortality (IHM). 
In our ablation study, we want to find out 
- if the inclusion of imputation with UTDE improves the performance of the model.
- if the introduction of mTand improves the performance of the model.

Thus we plan to run training and evaluation on the below three control/test groups:

1. Control group
- Time series: "previous" imputation + mTand(ts) + UTDE gate
- Text: Textencoder + mTand(txt)
- multimodal fusion

2. Test1: Drop imputation by setting missing value to zero to prove the benefit of the imputation and UTDE approach.
- Time series: "zero" imputation + mTand(ts) + UTDE gate
- Text: Textencoder + mTand(txt)
- multimodal fusion
MULTCrossModel false true

3. Test2: Drop mTAND from processing to prove the impact of considering irregularity during embedding.
- Time series: "previous" imputation + UTDE gate
- Text: Textencoder
- multimodal fusion
 MULTCrossModel false false

### Evaluation metrics:
- AUPR
- AUROC
- F1

### Shared data and hyper parameter 
- 48 hour IHM data
- number of sample data: 1000
- training epoch: 6

Preprocess data

```
powershell .\decompress_mimic_data.ps1 -d data\MIMICIII_Original\mimic-iii-clinical-database-1.4 -o data\mimic3
powershell .\build_benchmark_data.ps1
powershell .\extract_med_notes.ps1
```

In [16]:
import sys

from tensorboardX import SummaryWriter

import warnings
import logging

logger = logging.getLogger(__name__)
from GlobalConfigs import *
from MultimodalMIMIC.model import *
from MultimodalMIMIC.train import *
from MultimodalMIMIC.checkpoint import *
from accelerate import Accelerator
from MultimodalMIMIC.interp import *

In [17]:
import pickle


def train_and_eval(args):
    if args.fp16:
        args.mixed_precision = "fp16"
    else:
        args.mixed_precision = "no"
    accelerator = Accelerator(mixed_precision=args.mixed_precision, cpu=args.cpu)

    device = accelerator.device
    print(f'device: {device}')
    os.makedirs(args.output_dir, exist_ok=True)
    if args.tensorboard_dir != None:
        writer = SummaryWriter(args.tensorboard_dir)
    else:
        writer = None

    warnings.filterwarnings('ignore')
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )

    if args.seed is not None:
        set_seed(args.seed)

    output_path = make_save_dir(args)

    if args.seed == 0:
        copy_file(args.ck_file_path + 'model/', src=os.getcwd())

    # load data

    from MultimodalMIMIC.data import data_perpare
    from MultimodalMIMIC.util import loadBert

    if args.mode == 'train':
            if 'Text' in args.modeltype:
                BioBert, BioBertConfig, tokenizer = loadBert(args, device)
            else:
                BioBert, tokenizer = None, None
            train_dataset, train_sampler, train_dataloader = data_perpare(args, 'train', tokenizer)
            val_dataset, val_sampler, val_dataloader = data_perpare(args, 'val', tokenizer)
            _, _, test_data_loader = data_perpare(args, 'test', tokenizer)

    # load model
    if 'Text' in args.modeltype:
        model = MULTCrossModel(args=args, device=device, orig_d_ts=17, orig_reg_d_ts=34, orig_d_txt=768,
                            ts_seq_num=args.tt_max, text_seq_num=args.num_of_notes, Biobert=BioBert)
    else:
        model = TSMixed(args=args, device=device, orig_d_ts=17, orig_reg_d_ts=34, ts_seq_num=args.tt_max)

    # train model
    if args.modeltype == 'TS':
            optimizer = torch.optim.Adam(model.parameters(), lr=args.ts_learning_rate)
    elif args.modeltype == 'Text' or args.modeltype == 'TS_Text':
        optimizer = torch.optim.Adam([
            {'params': [p for n, p in model.named_parameters() if 'bert' not in n]},
            {'params': [p for n, p in model.named_parameters() if 'bert' in n], 'lr': args.txt_learning_rate}
        ], lr=args.ts_learning_rate)
    else:
        raise ValueError("Unknown modeltype in optimizer.")

    model, optimizer, train_dataloader, val_dataloader, test_data_loader = \
        accelerator.prepare(model, optimizer, train_dataloader, val_dataloader, test_data_loader)

    trainer_irg(model=model, args=args, accelerator=accelerator, train_dataloader=train_dataloader, \
                dev_dataloader=val_dataloader, test_data_loader=test_data_loader, device=device, \
                optimizer=optimizer, writer=writer)

    # eval model
    eval_test(args, model, test_data_loader, device)
    for result_file in os.listdir(args.ck_file_path):
        if 'result.pkl' in result_file:
            eval_result_path = args.ck_file_path + result_file
            # print(eval_result_path)
            with open(eval_result_path,'rb') as f:
                evaluation_result = pickle.load(f)
                print(evaluation_result)

### Control

Train and evaluate model follow the original set up from paper.
- Time series: "previous" imputation and mTand(ts), combined with UTDE
- Clinical notes: text encoder + mTand(txt)
- Combined result of above two with MultiModal fusion 

In [18]:
# pre-processed data with previous imputation
# set arguments
from MultimodalMIMIC.util import parse_args
ihm_discrete_save_path_control = f'{MULTI_MODAL_MIMIC_PATH}/Data/ihm_1000_previous'

parser = parse_args()
args = parser.parse_args(['--num_train_epochs','6',
                         '--train_batch_size','2',
                         '--eval_batch_size','8',
                         '--gradient_accumulation_steps','16',
                         '--num_update_bert_epochs','2',
                         '--notes_order','Last',
                         '--max_length','512',
                         '--output_dir','run/TS_Text/control',
                         '--embed_dim','128',
                         '--model_name','bioLongformer',
                         '--file_path',f'{ihm_discrete_save_path_control}',
                         '--mixup_level','batch',
                         '--fp16',
                         '--irregular_learn_emb_text',
                         '--irregular_learn_emb_ts',
                         '--reg_ts'])


print(vars(args))

train_and_eval(args)



{'task': 'ihm', 'file_path': '/media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_previous', 'output_dir': 'run/TS_Text/control', 'tensorboard_dir': None, 'seed': 42, 'mode': 'train', 'modeltype': 'TS_Text', 'eval_score': ['auc', 'auprc', 'f1'], 'num_labels': 2, 'max_length': 512, 'pad_to_max_length': False, 'model_path': None, 'train_batch_size': 2, 'eval_batch_size': 8, 'num_update_bert_epochs': 2, 'num_train_epochs': 6, 'txt_learning_rate': 5e-05, 'ts_learning_rate': 0.0004, 'gradient_accumulation_steps': 16, 'weight_decay': 0.01, 'lr_scheduler_type': 'linear', 'pt_mask_ratio': 0.15, 'mean_mask_length': 3, 'chunk': False, 'chunk_type': 'sent_doc_pos', 'warmup_proportion': 0.1, 'kernel_size': 1, 'num_heads': 8, 'layers': 3, 'cross_layers': 3, 'embed_dim': 128, 'irregular_learn_emb_ts': True, 'irregular_learn_emb_text': True, 'reg_ts': True, 'tt_max': 48, 'embed_time': 64, 'ts_to_txt': False, 'txt_to_ts': False, 'dropout': 0.1, 'model_name': 'bioLongformer

Some weights of LongformerModel were not initialized from the model checkpoint at yikuan8/Clinical-Longformer and are newly initialized: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Franco Trying to load: /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_previous/trainp2x_data.pkl
Using /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_previous/trainp2x_data.pkl
Franco Trying to load: /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_previous/valp2x_data.pkl
Using /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_previous/valp2x_data.pkl
Franco Trying to load: /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_previous/testp2x_data.pkl
Using /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_previous/testp2x_data.pkl


  0%|          | 0/6 [00:00<?, ?it/s]

bert update at epoch 0
0 True


...

Current auc 0.873964424951267
Best auc 0.873964424951267
Current auprc 0.5867509553646333
Best auprc 0.5867509553646333
Current f1 0.4653465346534653
Best f1 0.5161290322580645
5 False



0it [00:00, ?it/s]
1it [00:00,  4.27it/s]
2it [00:00,  4.43it/s]
3it [00:00,  4.32it/s]
4it [00:00,  4.34it/s]
5it [00:01,  4.40it/s]Input ids are automatically padded from 228 to 512 to be a multiple of `config.attention_window`: 512

6it [00:01,  4.41it/s]
7it [00:01,  4.42it/s]
8it [00:01,  4.45it/s]
9it [00:02,  4.46it/s]
10it [00:02,  4.42it/s]
11it [00:02,  4.39it/s]
12it [00:02,  4.39it/s]
13it [00:02,  4.41it/s]
14it [00:03,  4.41it/s]
15it [00:03,  4.42it/s]
16it [00:03,  4.40it/s]
17it [00:03,  4.40it/s]
18it [00:04,  4.39it/s]
19it [00:04,  4.39it/s]
20it [00:04,  4.40it/s]
21it [00:04,  4.38it/s]
22it [00:04,  4.42it/s]
23it [00:05,  4.43it/s]
24it [00:05,  4.40it/s]
25it [00:05,  4.44it/s]
26it [00:05,  4.44it/s]
27it [00:06,  4.48it/s]
28it [00:06,  4.51it/s]
29it [00:06,  4.55it/s]
30it [00:06,  4.49it/s]
31it [00:07,  4.49it/s]
32it [00:07,  4.42it/s]
33it [00:07,  4.34it/s]
34it [00:07,  5.01it/s]
35it [00:07,  4.78it/s]
36it [00:08,  4.70it/s]
37it [00:08,  4.58it/s]

Current auc 0.8751305346700083
Best auc 0.8751305346700083
Current auprc 0.5943903244056398
Best auprc 0.5943903244056398
Current f1 0.4901960784313725
Best f1 0.5161290322580645
run/TS_Text/control/ihm/TS_Text/TS_48/Atten/Text_48/bioLongformer/512/cross_attn3/irregular_TS_64/irregular_Text_64/5e-05_2_3_0.0004_6_8_128_1_2/f1/42.pth.tar



100%|██████████| 125/125 [01:31<00:00,  1.37it/s]

{42: {'auc': {'val': 0.8536619326093009, 'test': 0.8252864782276546}, 'auprc': {'val': 0.5274366693406054, 'test': 0.41081251193680945}, 'f1': {'val': 0.5161290322580645, 'test': 0.4358974358974359}}}


### Test1: Drop imputation by setting missing value to zero to prove the benefit of the imputation and UTDE approach.

Train and evaluate model dropping "previous" data imputation by setting "imputation_strategy" to "zero"
- Time series: *"zero"* imputation and mTand(ts), combined with UTDE
- Clinical notes: text encoder + mTand(txt)
- Combined result of above two with MultiModal fusion

In [20]:
# pre-processed data with zero imputation

# set arguments
from MultimodalMIMIC.util import parse_args
ihm_discrete_save_path_test1 = f'{MULTI_MODAL_MIMIC_PATH}/Data/ihm_1000_zero'
ihm_eval_output_test1 = 'run/TS_Text/test1'

parser = parse_args()
args_zero = parser.parse_args(['--num_train_epochs','6',
                         '--train_batch_size','2',
                         '--eval_batch_size','8',
                         '--gradient_accumulation_steps','16',
                         '--num_update_bert_epochs','2',
                         '--notes_order','Last',
                         '--max_length','512',
                         '--output_dir',ihm_eval_output_test1,
                         '--embed_dim','128',
                         '--model_name','bioLongformer',
                         '--file_path',f'{ihm_discrete_save_path_test1}',
                         '--mixup_level','batch',
                         '--fp16',
                         '--irregular_learn_emb_text',
                         '--irregular_learn_emb_ts',
                         '--reg_ts'])


print(vars(args_zero))

train_and_eval(args_zero)


{'task': 'ihm', 'file_path': '/media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_zero', 'output_dir': 'run/TS_Text/test1', 'tensorboard_dir': None, 'seed': 42, 'mode': 'train', 'modeltype': 'TS_Text', 'eval_score': ['auc', 'auprc', 'f1'], 'num_labels': 2, 'max_length': 512, 'pad_to_max_length': False, 'model_path': None, 'train_batch_size': 2, 'eval_batch_size': 8, 'num_update_bert_epochs': 2, 'num_train_epochs': 6, 'txt_learning_rate': 5e-05, 'ts_learning_rate': 0.0004, 'gradient_accumulation_steps': 16, 'weight_decay': 0.01, 'lr_scheduler_type': 'linear', 'pt_mask_ratio': 0.15, 'mean_mask_length': 3, 'chunk': False, 'chunk_type': 'sent_doc_pos', 'warmup_proportion': 0.1, 'kernel_size': 1, 'num_heads': 8, 'layers': 3, 'cross_layers': 3, 'embed_dim': 128, 'irregular_learn_emb_ts': True, 'irregular_learn_emb_text': True, 'reg_ts': True, 'tt_max': 48, 'embed_time': 64, 'ts_to_txt': False, 'txt_to_ts': False, 'dropout': 0.1, 'model_name': 'bioLongformer', 'nu

Some weights of LongformerModel were not initialized from the model checkpoint at yikuan8/Clinical-Longformer and are newly initialized: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Franco Trying to load: /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_zero/trainp2x_data.pkl
Using /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_zero/trainp2x_data.pkl
Franco Trying to load: /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_zero/valp2x_data.pkl
Using /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_zero/valp2x_data.pkl
Franco Trying to load: /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_zero/testp2x_data.pkl
Using /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_zero/testp2x_data.pkl


  0%|          | 0/6 [00:00<?, ?it/s]

bert update at epoch 0
0 True


...

Current auc 0.873964424951267
Best auc 0.873964424951267
Current auprc 0.5867509553646333
Best auprc 0.5867509553646333
Current f1 0.4653465346534653
Best f1 0.5161290322580645
5 False



0it [00:00, ?it/s]
1it [00:00,  4.24it/s]
2it [00:00,  4.44it/s]
3it [00:00,  4.32it/s]
4it [00:00,  4.34it/s]
5it [00:01,  4.39it/s]
6it [00:01,  4.38it/s]
7it [00:01,  4.39it/s]
8it [00:01,  4.42it/s]
9it [00:02,  4.45it/s]
10it [00:02,  4.41it/s]
11it [00:02,  4.39it/s]
12it [00:02,  4.41it/s]
13it [00:02,  4.42it/s]
14it [00:03,  4.40it/s]
15it [00:03,  4.43it/s]
16it [00:03,  4.43it/s]
17it [00:03,  4.46it/s]
18it [00:04,  4.44it/s]
19it [00:04,  4.47it/s]
20it [00:04,  4.45it/s]
21it [00:04,  4.43it/s]
22it [00:04,  4.46it/s]
23it [00:05,  4.47it/s]
24it [00:05,  4.44it/s]
25it [00:05,  4.46it/s]
26it [00:05,  4.44it/s]
27it [00:06,  4.41it/s]
28it [00:06,  4.46it/s]
29it [00:06,  4.49it/s]
30it [00:06,  4.42it/s]
31it [00:07,  4.44it/s]
32it [00:07,  4.39it/s]
33it [00:07,  4.32it/s]
34it [00:07,  4.98it/s]
35it [00:07,  4.77it/s]
36it [00:08,  4.69it/s]
37it [00:08,  4.58it/s]
38it [00:08,  4.55it/s]
39it [00:08,  4.53it/s]
40it [00:08,  4.52it/s]
41it [00:09,  4.51it/s]
42it 

Current auc 0.8751305346700083
Best auc 0.8751305346700083
Current auprc 0.5943903244056398
Best auprc 0.5943903244056398
Current f1 0.4901960784313725
Best f1 0.5161290322580645
run/TS_Text/test1/ihm/TS_Text/TS_48/Atten/Text_48/bioLongformer/512/cross_attn3/irregular_TS_64/irregular_Text_64/5e-05_2_3_0.0004_6_8_128_1_2/f1/42.pth.tar


100%|██████████| 125/125 [01:31<00:00,  1.36it/s]

{42: {'auc': {'val': 0.8536619326093009, 'test': 0.8252864782276546}, 'auprc': {'val': 0.5274366693406054, 'test': 0.41081251193680945}, 'f1': {'val': 0.5161290322580645, 'test': 0.4358974358974359}}}


### Test2: Drop mTAND from processing to prove the impact of considering irregularity during embedding.

Train and evaluate model dropping "mTand" module.
- Time series: "previous" imputation
- Clinical notes: text encoder
- Combined result of above two with MultiModal fusion

Drop mTand by set these arguments:
- 'irregular_learn_emb_ts': False, 
- 'irregular_learn_emb_text': False

In [21]:
ihm_discrete_save_path_test2 = f'{MULTI_MODAL_MIMIC_PATH}/Data/ihm/ihm_1000_samples'
ihm_eval_output_test2 = 'run/TS_Text/test2'

parser = parse_args()
args_test2 = parser.parse_args(['--num_train_epochs','6',
                         '--train_batch_size','2',
                         '--eval_batch_size','8',
                         '--gradient_accumulation_steps','16',
                         '--num_update_bert_epochs','2',
                         '--notes_order','Last',
                         '--max_length','512',
                         '--output_dir',ihm_eval_output_test2,
                         '--embed_dim','128',
                         '--model_name','bioLongformer',
                         '--file_path',f'{ihm_discrete_save_path_control}',
                         '--mixup_level','batch',
                         '--fp16',
                         '--reg_ts'])


print(vars(args_test2))

train_and_eval(args_test2)

{'task': 'ihm', 'file_path': '/media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_previous', 'output_dir': 'run/TS_Text/test2', 'tensorboard_dir': None, 'seed': 42, 'mode': 'train', 'modeltype': 'TS_Text', 'eval_score': ['auc', 'auprc', 'f1'], 'num_labels': 2, 'max_length': 512, 'pad_to_max_length': False, 'model_path': None, 'train_batch_size': 2, 'eval_batch_size': 8, 'num_update_bert_epochs': 2, 'num_train_epochs': 6, 'txt_learning_rate': 5e-05, 'ts_learning_rate': 0.0004, 'gradient_accumulation_steps': 16, 'weight_decay': 0.01, 'lr_scheduler_type': 'linear', 'pt_mask_ratio': 0.15, 'mean_mask_length': 3, 'chunk': False, 'chunk_type': 'sent_doc_pos', 'warmup_proportion': 0.1, 'kernel_size': 1, 'num_heads': 8, 'layers': 3, 'cross_layers': 3, 'embed_dim': 128, 'irregular_learn_emb_ts': False, 'irregular_learn_emb_text': False, 'reg_ts': True, 'tt_max': 48, 'embed_time': 64, 'ts_to_txt': False, 'txt_to_ts': False, 'dropout': 0.1, 'model_name': 'bioLongformer

Some weights of LongformerModel were not initialized from the model checkpoint at yikuan8/Clinical-Longformer and are newly initialized: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Franco Trying to load: /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_previous/trainp2x_data.pkl
Using /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_previous/trainp2x_data.pkl
Franco Trying to load: /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_previous/valp2x_data.pkl
Using /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_previous/valp2x_data.pkl
Franco Trying to load: /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_previous/testp2x_data.pkl
Using /media/ftrujillo/FRD/Projects/UIUC/DLH/CS598_Final/MultimodalMIMIC/Data/ihm_1000_previous/testp2x_data.pkl


  0%|          | 0/6 [00:00<?, ?it/s]

bert update at epoch 0
0 True


...

Current auc 0.9255212684488999
Best auc 0.9255212684488999
Current auprc 0.7274338142014949
Best auprc 0.7274338142014949
Current f1 0.43678160919540227
Best f1 0.5304347826086956
5 False



0it [00:00, ?it/s]
1it [00:00,  4.04it/s]
2it [00:00,  4.35it/s]
3it [00:00,  4.37it/s]
4it [00:00,  4.41it/s]
5it [00:01,  4.42it/s]
6it [00:01,  4.46it/s]
7it [00:01,  4.43it/s]
8it [00:01,  4.40it/s]
9it [00:02,  4.43it/s]
10it [00:02,  4.46it/s]
11it [00:02,  4.50it/s]
12it [00:02,  4.51it/s]
13it [00:02,  4.51it/s]
14it [00:03,  4.51it/s]
15it [00:03,  4.52it/s]
16it [00:03,  4.50it/s]
17it [00:03,  4.50it/s]
18it [00:04,  4.46it/s]
19it [00:04,  4.45it/s]
20it [00:04,  4.51it/s]
21it [00:04,  4.50it/s]
22it [00:04,  4.50it/s]
23it [00:05,  4.55it/s]
24it [00:05,  4.55it/s]
25it [00:05,  4.53it/s]
26it [00:05,  4.49it/s]
27it [00:06,  4.49it/s]
28it [00:06,  4.49it/s]
29it [00:06,  4.50it/s]
30it [00:06,  4.47it/s]
31it [00:06,  4.42it/s]
32it [00:07,  4.42it/s]
33it [00:07,  4.45it/s]
34it [00:07,  4.49it/s]
35it [00:07,  4.53it/s]
36it [00:08,  4.47it/s]
37it [00:08,  4.45it/s]
38it [00:08,  4.51it/s]
39it [00:08,  4.51it/s]
40it [00:08,  4.57it/s]
41it [00:09,  4.57it/s]
42it 

Current auc 0.9383528265107213
Best auc 0.9383528265107213
Current auprc 0.7663949217679294
Best auprc 0.7663949217679294
Current f1 0.6388888888888888
Best f1 0.6388888888888888
run/TS_Text/test2/ihm/TS_Text/cross_attn3/regular_TS/regular_Text/5e-05_2_3_0.0004_6_8_128_1_2/f1/42.pth.tar


100%|██████████| 125/125 [01:33<00:00,  1.33it/s]

{42: {'auc': {'val': 0.9383528265107213, 'test': 0.7571571810542399}, 'auprc': {'val': 0.7663949217679294, 'test': 0.2585484144905148}, 'f1': {'val': 0.6388888888888888, 'test': 0.23204419889502764}}}


### Ablation result comparison


| test group    |  test accuracy |test AUPR | test F1 |
|---|---|---|---|
| control  | 0.8216|0.4050|0.4323|
| test1: Drop imputation and UTDE |0.8216|0.4050|0.4323|
| Test2: Drop mTAND from processing    | 0.7154|0.2359|0.2772|

### Ablation conclusion

Given the above comparison, we can see that 
 - the usage of imputation, UTDE ???
 - the usage of mTand did contribute to the improvement of the model performance.